In [ ]:
# import sys
# !{sys.executable} -m pip install matplotlib --user
# !{sys.executable} -m pip install hyperopt --user

In [ ]:
from time import time
from hyperopt import tpe, fmin, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from copy import deepcopy
from sklearn.neural_network import MLPClassifier
import numpy as np

# Data science imports
import pandas as pd
from enum import Enum
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler, Imputer, LabelBinarizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn_pandas import DataFrameMapper

# Visualisation imports
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# File IO: Read the data file
file = open('train_data.txt', mode = 'r')
lines = file.readlines()
file.close()

In [ ]:
# Check for completeness
len(lines)

In [ ]:
# Pre-processing: Clean unwanted characters 
data = []
i = 0
while i < len(lines)-1:
    row = {}
    row['id'] = lines[i+1].split(" ")[1].strip()
    row['durationOfStay'] = lines[i+2].split(" ")[1].strip()
    row['gender'] = lines[i+3].split(" ")[1].strip()
    row['Age'] = lines[i+4].split(" ")[1].strip()
    row['kids'] = lines[i+5].split(" ")[1].strip()
    row['destinationCode'] = lines[i+6].split(" ")[1].strip()
    row['AcomType'] = lines[i+7].split(" ")[1].strip()
    data.append(row)
    i = i + 8

In [ ]:
# Structure the data
def structure_data_to_df(data, test_data=False):
    if test_data:
        df = pd.DataFrame(data, columns=['id','durationOfStay','gender','Age','kids','destinationCode'])
        df['id'] = df['id'].astype(str)
    else:
        df = pd.DataFrame(data, columns=['durationOfStay','gender','Age','kids','destinationCode','AcomType'])
        df['AcomType'] = df['AcomType'].astype(str)
        
    df['gender'] = df['gender'].astype(str)
    df['kids'] = df['kids'].astype(str)
    df['destinationCode'] = df['destinationCode'].astype(str)
    df = df.replace('<NA>', np.nan)
    df = df.replace('NA', np.nan)
    df = df.replace('nan', np.nan)
    df[['durationOfStay', 'Age']] = df[['durationOfStay', 'Age']].apply(pd.to_numeric)
    
    return df
    
df = structure_data_to_df(data)

In [ ]:
# utility variables
categorical_cols = ['gender']
missing_categorical_cols = ['kids', 'destinationCode']
numerical_cols = ['durationOfStay']
missing_numerical_cols = ['Age']

# transform utility function
def build_imputer_and_transformer(missing_categorical_cols, categorical_cols, 
                                  missing_numerical_cols, numerical_cols, constant=True):
    transformations = []
    
    for col in missing_categorical_cols:
        if constant:
            transformations.append(([col], [SimpleImputer(strategy='constant', fill_value='N/A'), LabelBinarizer()]))
        else:
            transformations.append(([col], [SimpleImputer(strategy='most_frequent'), LabelBinarizer()]))
        
    for col in categorical_cols:
        transformations.append((col, LabelBinarizer()))
        
    transformations.append((missing_numerical_cols, [SimpleImputer(strategy='mean'), StandardScaler()]))
    transformations.append((numerical_cols, StandardScaler()))
   
    transformer = DataFrameMapper(transformations)
    return transformer

In [ ]:
y = df['AcomType'].copy()
X = df.drop(['AcomType'], axis=1)
    
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=0)
    
transformer = build_imputer_and_transformer(missing_categorical_cols, categorical_cols,
                                             missing_numerical_cols, numerical_cols, constant=True)
    
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

In [ ]:
hidden_layer_sizes_list =[(20, 30), (30, 10), (10, 20), (5,10)]
# hidden_layer_sizes_list =[(200, 100), (100, 200), (150, 300), (300, 100), (100, 200, 100), (200,100,200)]
activation_list = ['identity', 'logistic', 'tanh', 'relu']
solver_list = ['lbfgs','sgd', 'adam']
learning_rate_list = ['constant','adaptive', 'invscaling']
batch_size_list = np.arange(20, 200, dtype=int)
max_iter_list = np.arange(20, 500, dtype=int)

In [ ]:
def optimize(evals, trials, optimizer=tpe.suggest):
    space = {
        'hidden_layer_sizes': hp.choice('hidden_layer_sizes', hidden_layer_sizes_list),
        'activation': hp.choice('activation', activation_list),
        'solver': hp.choice('solver', solver_list),
        'alpha': hp.lognormal('alpha', 0, 1), # [0.0001, 0.05],
#         'alpha': hp.loguniform('alpha', np.log(0.1), np.log(10)),
        'learning_rate': hp.choice('learning_rate', learning_rate_list),
        'batch_size': hp.choice('batch_size', batch_size_list),
        'max_iter': hp.choice('max_iter', max_iter_list),
    }
    best = fmin(score, space, algo=optimizer, max_evals=evals, trials=trials)
    pbar.close()
    return best

In [ ]:
def score(params):
     
    model = MLPClassifier(
            hidden_layer_sizes = params['hidden_layer_sizes'],
            activation = params['activation'],
            solver = params['solver'],
            alpha = params['alpha'],
            learning_rate = params['learning_rate'],
            batch_size = params['batch_size'],
            max_iter = params['max_iter'],
            shuffle = True,
            random_state = 42,
            warm_start = False,
            early_stopping = True,
            validation_fraction = 0.1,
    )

    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='balanced_accuracy', n_jobs=40)  
    
    scores_mean = 1-scores.mean()

    
    print('Parameters with this training score {} :'.format(scores_mean))
    print(params)
    pbar.update()
#     return {'loss': test_loss, 'status': STATUS_OK}
    return {'loss': scores_mean, 'status': STATUS_OK}

In [ ]:
import time
from tqdm import tqdm

trials = Trials()
cores = 48
start = time.time()
evaluations = 1000
pbar = tqdm(total=evaluations, desc="Hyperopt")
best_param = optimize(evals=evaluations,
                      optimizer=tpe.suggest,
                      trials=trials)
print("------------------------------------")
print("The best hyperparameters are: ", "\n")
print(best_param)
end = time.time()
print('Time elapsed to optimize {0} executions: {1}'.format(evaluations, end - start))
best_param['hidden_layer_sizes'] = hidden_layer_sizes_list[best_param['hidden_layer_sizes']]
best_param['activation'] = activation_list[best_param['activation']]
best_param['solver'] = solver_list[best_param['solver']]
best_param['learning_rate'] = learning_rate_list[best_param['learning_rate']]
best_param['batch_size'] = batch_size_list[best_param['batch_size']]
best_param['max_iter'] = max_iter_list[best_param['max_iter']]
print('\n Best score:')
score(best_param)

In [ ]:
# get the best params and train the model
mlp_opt = MLPClassifier(
            hidden_layer_sizes = best_param['hidden_layer_sizes'],
            activation = best_param['activation'],
            solver = best_param['solver'],
            alpha = best_param['alpha'],
            learning_rate = best_param['learning_rate'],
            batch_size = best_param['batch_size'],
            max_iter = best_param['max_iter'],
            shuffle = True,
            random_state = 42,
            warm_start = False,
            early_stopping = True,
            validation_fraction = 0.1,
    )

fitted_model = mlp_opt.fit(X=X_train,y=y_train)

In [ ]:
y_score = fitted_model.predict_proba(X_test)
y_score

In [ ]:
test_acc = accuracy_score(y_test, fitted_model.predict(X_test))
test_acc